In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
batch_size = 256
num_workers = 4
lr = 0.0005
num_epochs = 50

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

In [4]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2))
        self.fc = nn.Sequential(
            nn.Linear(256 * 5 * 5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        b, c, h, w = x.shape
        feature = self.conv(x)
        return self.fc(feature.view(b, -1))

In [5]:
trans = []
trans.append(transforms.Resize((224, 224)))
trans.append(transforms.ToTensor())
transform = transforms.Compose(trans)
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transform)
print(len(mnist_train), len(mnist_test))
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

60000 10000


In [6]:
net = AlexNet().cuda()
optimizer = torch.optim.Adam(net.parameters(), lr)
loss = torch.nn.CrossEntropyLoss()

In [7]:
def train_FashionMNIST(net, train_iter, optimizer):
    train_loss = 0.0
    train_acc = 0.0
    train_num = 0
    
    for X, y in train_iter:
        X = X.cuda()
        y = y.cuda()
        y_hat = net(X)
        l = loss(y_hat, y)
        optimizer.zero_grad()
        # net.zero_grad()
        l.backward()
        optimizer.step()
        
        train_loss += l.item()
        train_acc += (y_hat.argmax(dim=1) == y).sum().item()
        train_num += y.shape[0]
    
    train_loss /= train_num
    train_acc /= train_num
    print('train loss: %.4f, train acc: %.3f' % (train_loss, train_acc))

In [8]:
def test_FashionMNIST(net, test_iter):
    test_acc = 0.0
    test_num = 0
    
    for X, y in test_iter:
        X = X.cuda()
        y = y.cuda()
        y_hat = net(X)
        test_acc += (y_hat.argmax(dim=1) == y).sum().item()
        test_num += y.shape[0]
    
    test_acc /= test_num
    print('test acc: %.3f' % (test_acc))

In [9]:
for i in range(num_epochs):
    print(f'epoch: {i}')
    train_FashionMNIST(net, train_iter, optimizer)
    test_FashionMNIST(net, test_iter)
    print('----------------')

epoch: 0
train loss: 0.0027, train acc: 0.733
test acc: 0.851
----------------
epoch: 1
train loss: 0.0013, train acc: 0.873
test acc: 0.880
----------------
epoch: 2
train loss: 0.0011, train acc: 0.894
test acc: 0.892
----------------
epoch: 3
train loss: 0.0010, train acc: 0.909
test acc: 0.905
----------------
epoch: 4
train loss: 0.0009, train acc: 0.916
test acc: 0.913
----------------
epoch: 5
train loss: 0.0008, train acc: 0.924
test acc: 0.914
----------------
epoch: 6
train loss: 0.0007, train acc: 0.931
test acc: 0.912
----------------
epoch: 7
train loss: 0.0007, train acc: 0.935
test acc: 0.912
----------------
epoch: 8
train loss: 0.0006, train acc: 0.940
test acc: 0.916
----------------
epoch: 9
train loss: 0.0006, train acc: 0.945
test acc: 0.920
----------------
epoch: 10
train loss: 0.0005, train acc: 0.950
test acc: 0.921
----------------
epoch: 11
train loss: 0.0005, train acc: 0.954
test acc: 0.921
----------------
epoch: 12
train loss: 0.0004, train acc: 0.959
tes

In [10]:
# Adam: lr=0.001, epoch 20, test_acc=0.922, batch_size=256
# Adam: lr=0.005, epoch 19, test_acc=0.861, batch_size=256
# Adam: lr=0.0005, epoch 37, test_acc=0.928, batch_size=256